In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/casper-hansen/AutoAWQ

Cloning into 'AutoAWQ'...
remote: Enumerating objects: 2950, done.
remote: Counting objects: 100% (1430/1430), done.
remote: Compressing objects: 100% (685/685), done.
remote: Total 2950 (delta 1024), reused 909 (delta 727), pack-reused 1520
Receiving objects: 100% (2950/2950), 7.48 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (1826/1826), done.


In [ ]:
!cd AutoAWQ

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/zephyr-7B-beta-AWQ"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)
# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)

domains=[ 'economic events', 'social events', 'political unrest', 'elections', 'natural disasters', 'transport accidents', 'crimes', 'industrial accidents', 'international relations', 'home policy', 'MUC-like events', 'KBP-like events'

]
request = """
generate me 5 causal examples
"""

definition="""Below are the definitions for the tags in the sentence:
  Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>.

  Please generate causal sentences within this domains."""

for d in domains:


  prompt_template=f'''<|system|>
  </s>
  <|user|>
  {request + 'in the omain of ' +
    str(d) + definition }</s>
  <|assistant|>
  '''

  print("*** Running model.generate:")

  token_input = tokenizer(
      prompt_template,
      return_tensors='pt'
  ).input_ids.cuda()
  # Open a file named 'domain.txt' in write mode
  with open('domain.txt', 'w') as file:
  # Write the content of the string to the file
    file.write(prompt_template)

  # Generate output
  generation_output = model.generate(
      token_input,
      do_sample=True,
      temperature=0.7,
      top_p=0.95,
      top_k=40,
      max_new_tokens=512
  )

  # Get the tokens from the output, decode them, print them
  token_output = generation_output[0]
  text_output = tokenizer.decode(token_output)
  print("LLM output: ", text_output)

"""
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])
"""


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

eval_results.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 49.61it/s]


*** Running model.generate:
LLM output:  <s> <|system|>
  </s> 
  <|user|>
  
generate me 5 causal examples
in the omain of economic eventsBelow are the definitions for the tags in the sentence:
  Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>.

  Please generate causal sentences within this domains.</s> 
  <|assistant|>
  1. Cause: Increase in minimum wage laws. Effect: Higher labor costs for businesses. Signal: Therefore. Example: Increasing the minimum wage will result in higher labor costs for businesses, therefore, some small businesses may struggle to remain profitable.

2. Cause: Decrease in interest rates. Effect: Increased demand for borrowing. Signal: As a result. Example: Lowering interest rates will lead to an increased demand for borrowing, as a r

'\n# Inference should be possible with transformers pipeline as well in future\n# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)\nfrom transformers import pipeline\n\nprint("*** Pipeline:")\npipe = pipeline(\n    "text-generation",\n    model=model,\n    tokenizer=tokenizer,\n    max_new_tokens=512,\n    do_sample=True,\n    temperature=0.7,\n    top_p=0.95,\n    top_k=40,\n    repetition_penalty=1.1\n)\n\nprint(pipe(prompt_template)[0][\'generated_text\'])\n'

In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "TheBloke/zephyr-7B-beta-AWQ"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)
# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)

domains = ['economic events', 'social events', 'political unrest', 'elections', 'natural disasters', 'transport accidents', 'crimes', 'industrial accidents', 'international relations', 'home policy', 'MUC-like events', 'KBP-like events']

request = """
generate me 5 causal examples
"""

definition = """Below are the definitions for the tags in the sentence:
  Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>.

  Please generate causal sentences within this domains."""

# Open a file named 'domain.txt' in write mode
with open('domain.txt', 'w') as file:
    for d in domains:
        prompt_template = f'''
        </s>

        {request + 'in the domain of ' +
            str(d) + definition}</s>

        '''

        print("*** Running model.generate:")

        token_input = tokenizer(
            prompt_template,
            return_tensors='pt'
        ).input_ids.cuda()

        # Generate output
        generation_output = model.generate(
            token_input,
            do_sample=True,
            temperature=0.7,
            top_p=0.95,
            top_k=40,
            max_new_tokens=512
        )

        # Get the tokens from the output, decode them, print them
        token_output = generation_output[0]
        text_output = tokenizer.decode(token_output)
        print("LLM output: ", text_output)

        # Write the generated text to the file
        file.write(text_output + '\n')

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Fusing layers...: 100%|██████████| 32/32 [00:00<00:00, 53.69it/s]


*** Running model.generate:
LLM output:  <s> 
        </s> 

        
generate me 5 causal examples
in the domain of economic eventsBelow are the definitions for the tags in the sentence:
  Cause: The reason for an event happening, to be enclosed between <ARG0> and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between <SIG0> and </SIG0>.

  Please generate causal sentences within this domains.</s> 

        
<ARG0>A decrease in interest rates</ARG0>, <SIG0>leads to</SIG0> <ARG1>increased borrowing</ARG1> among consumers and businesses, which in turn can stimulate economic growth.

<ARG0>Government subsidies for research and development</ARG0>, <SIG0>encourage</SIG0> <ARG1>innovation</ARG1> in industries, resulting in increased productivity and economic competitiveness.

<ARG0>A sudden drop in the value of a country's currency</ARG0>, <SIG0>causes</SIG0> <ARG1>

load the txt file, concatenat the examples in a csv fil.


In [ ]:
import csv
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Function to extract causal relations from a sentence
def extract_causal_relations(sentence):
    # Placeholder function, you would implement this according to your specific causal relation extraction method
    # For demonstration purposes, let's assume no causal relations are extracted
    return []

# Function to write data to CSV file
def write_to_csv(data, output_filename):
    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text', 'num_rs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Load causal text file
with open('/content/domain.txt', 'r') as file:
    causal_text = file.read()

# Tokenize into sentences
sentences = sent_tokenize(causal_text)

# Initialize data list to hold rows for CSV
csv_data = []

# Process each sentence
for sent_id, sentence in enumerate(sentences, start=1):
    causal_relations = extract_causal_relations(sentence)
    eg_id = 0
    for index, causal_text_w_pairs in enumerate(causal_relations, start=1):
        eg_id += 1
        num_rs = len(causal_text_w_pairs)
        row = {
            'corpus': 'Your Corpus Name',
            'doc_id': 'Your Document Name',
            'sent_id': sent_id,
            'eg_id': eg_id,
            'index': f'{sent_id}_{eg_id}',
            'text': sentence,
            'causal_text': causal_text_w_pairs,
            'num_rs': num_rs
        }
        csv_data.append(row)

# Write data to CSV file
write_to_csv(csv_data, '/content/train_subtask2_grouped.csv')

print("CSV file saved successfully.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CSV file saved successfully.


In [ ]:
import csv
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Function to extract causal relations from a sentence
def extract_causal_relations(sentence):
    # Placeholder function, you would implement this according to your specific causal relation extraction method
    # For demonstration purposes, let's assume no causal relations are extracted
    return []

# Function to write data to CSV file
def write_to_csv(data, output_filename):
    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text', 'num_rs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

# Load causal text file
with open('/content/domain.txt', 'r') as file:
    causal_text = file.read()

# Tokenize into sentences
sentences = sent_tokenize(causal_text)

# Initialize data list to hold rows for CSV
csv_data = []

# Process each sentence
for sent_id, sentence in enumerate(sentences, start=1):
    causal_relations = extract_causal_relations(sentence)
    eg_id = 0
    for index, causal_text_w_pairs in enumerate(causal_relations, start=1):
        eg_id += 1
        num_rs = len(causal_text_w_pairs)
        row = {
            'corpus': 'Your Corpus Name',
            'doc_id': 'Your Document Name',
            'sent_id': sent_id,
            'eg_id': eg_id,
            'index': f'{sent_id}_{eg_id}',
            'text': sentence,
            'causal_text': causal_text_w_pairs,
            'num_rs': num_rs
        }
        csv_data.append(row)

# Write data to CSV file
write_to_csv(csv_data, '/content/train_subtask2_grouped.csv')

print("CSV file saved successfully.")

CSV file saved successfully.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import csv

# Function to extract causal examples and format them
def extract_causal_examples(text):
    examples = []
    domain_start = 0
    while True:
        domain_start = text.find("<ARG0>", domain_start)
        if domain_start == -1:
            break

        domain_end = text.find("</s>", domain_start)
        domain_text = text[domain_start:domain_end]

        cause_start = domain_text.find("<ARG0>") + len("<ARG0>")
        cause_end = domain_text.find("<SIG0>")
        cause = domain_text[cause_start:cause_end].strip()

        signal_start = cause_end + len("<SIG0>")
        signal_end = domain_text.find("</SIG0>")
        signal = domain_text[signal_start:signal_end].strip()

        effect_start = signal_end + len("</SIG0>")
        effect_end = domain_text.find("</ARG1>")
        effect = domain_text[effect_start:effect_end].strip()

        examples.append({
            'text': cause + ' ' + signal + ' ' + effect,
            'causal_text': [cause, signal, effect],
            'num_rs': 3
        })

        domain_start = domain_end + len("</s>")

    return examples

# Function to write data to CSV file
def write_to_csv(data, output_filename):
    with open(output_filename, 'w', newline='') as csvfile:
        fieldnames = ['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text', 'num_rs']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# Read the text file
with open('/content/domain.txt', 'r') as file:
    text = file.read()

# Extract causal examples
examples = extract_causal_examples(text)

# Initialize data list to hold rows for CSV
csv_data = []

# Add examples to data list
for i, example in enumerate(examples, start=1):
    csv_data.append({
        'corpus': 'Your Corpus Name',
        'doc_id': f'Document {i}',
        'sent_id': 1,
        'eg_id': i,
        'index': f'{i}_1',
        'text': example['text'],
        'causal_text': example['causal_text'],
        'num_rs': example['num_rs']
    })

# Write data to CSV file
write_to_csv(csv_data, '/content/train_subtask2_grouped.csv')

print("CSV file saved successfully.")

CSV file saved successfully.


In [ ]:
import csv

# Load the text file
with open('domain.txt', 'r') as file:
    examples = file.readlines()

# Concatenate the examples into a single string
concatenated_examples = ' '.join(examples)

# Split the concatenated examples by domain
split_examples = concatenated_examples.split('*** Running model.generate:')

# Remove the empty first element
split_examples = split_examples[1:]

# Open a file named 'examples.csv' in write mode
with open('examples.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([domains, text_output])

    for domain in domains:
        # Find the example corresponding to the current domain
        for example in split_examples:
            if f"in the domain of {domain}" in example:
                # Extract the generated output for this domain
                _, generated_output = example.split('\n', 1)
                generated_output = generated_output.strip()

                # Write domain name and generated output to CSV file
                writer.writerow([domain, generated_output])
                break  # Stop searching for this domain once found

prompt using the examples you have in your csv. try injecting 2, 3, 5 examples at a time and lets observe the difference by mnaual annotation of the answers


In [ ]:
from transformers import pipeline
import pandas as pd

# Load examples from the CSV file
examples_df = pd.read_csv('/content/train_subtask2_grouped.csv')

# Create a function to generate prompts with varying numbers of examples
def generate_prompt(examples, num_examples):
    # Select the specified number of examples
    selected_examples = examples[:num_examples]
    prompt = "\n\n".join(selected_examples)
    return prompt

# Function to generate and print outputs for different number of examples
def generate_outputs(examples_df, num_examples):
    prompt = generate_prompt(examples_df['causal_text'].tolist(), num_examples)
    print("Prompt:\n", prompt)
    print("\n*** Generating output with", num_examples, "examples:")
    output = generator(prompt)
    print("Generated output:\n", output[0]['generated_text'])
    print("--------------------------------------------------\n")

# Load the text generation pipeline
generator = pipeline("text-generation")

# Generate outputs for 2, 3, and 5 examples
generate_outputs(examples_df, 2)
#generate_outputs(examples_df, 3)
#generate_outputs(examples_df, 5)

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:
 ['and </ARG0>.\n  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.\n  Signal: Words that transition the cause to the effect, to be enclosed between', 'and', '']

['A decrease in interest rates</ARG0>,', 'leads to', '<ARG1>increased borrowing']

*** Generating output with 2 examples:


ValueError: Input length of input_ids is 89, but `max_length` is set to 50. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/train_subtask2_grouped.csv')

# Function to inject examples and observe manual annotations
def inject_and_observe(num_examples):
    # Inject examples
    injected_examples = df.head(num_examples)

    # Print injected examples
    print(f"\nInjected Examples ({num_examples} examples):")
    for index, row in injected_examples.iterrows():
        print(f"Example {index + 1}:")
        print(f"Text: {row['text']}")
        print(f"Causal Text: {row['causal_text']}")
        print()  # Add a newline for better readability

    # Here you would manually annotate the injected examples and observe the differences

# Inject and observe 2 examples
inject_and_observe(2)

# Inject and observe 3 examples
inject_and_observe(3)

# Inject and observe 5 examples
inject_and_observe(5)


Injected Examples (2 examples):
Example 1:
Text: and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between and 
Causal Text: ['and </ARG0>.\n  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.\n  Signal: Words that transition the cause to the effect, to be enclosed between', 'and', '']

Example 2:
Text: A decrease in interest rates</ARG0>, leads to <ARG1>increased borrowing
Causal Text: ['A decrease in interest rates</ARG0>,', 'leads to', '<ARG1>increased borrowing']


Injected Examples (3 examples):
Example 1:
Text: and </ARG0>.
  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </ARG1>.
  Signal: Words that transition the cause to the effect, to be enclosed between and 
Causal Text: ['and </ARG0>.\n  Effect: The event that occurs due to the cause, to be enclosed between <ARG1> and </A

In [ ]:
#Concatenation of the previous dataset and the new dataset
# Load the previous dataset
import pandas as pd
previous_df = pd.read_csv('/content/drive/MyDrive/Semester Project/Data/train_subtask2_grouped.csv')
print(previous_df.shape)

# Load the new dataset
new_df = pd.read_csv('/content/train_subtask2_grouped.csv')

# Concatenate the previous and new datasets
concatenated_df = pd.concat([previous_df, new_df], ignore_index=True)

# Save the concatenated dataset to a new CSV file
concatenated_df.to_csv('concatenated_dataset.csv', index=False)
print(concatenated_df.shape)

(3075, 8)
(3098, 9)


Checking manually if there's a causal relation in the text:


In [ ]:
import pandas as pd

def manual_check_causal_relation(df):
    for index, row in df.iterrows():
        print(f"\nExample {index + 1}:")
        print(f"Text: {row['text']}")
        causal_relation = input("Does this example contain a causal relation? (Yes/No): ").strip().lower()
        while causal_relation not in ['yes', 'no']:
            print("Invalid input. Please enter 'Yes' or 'No'.")
            causal_relation = input("Does this example contain a causal relation? (Yes/No): ").strip().lower()
        df.at[index, 'causal_relation'] = causal_relation

    return df

# Load the dataset
df = pd.read_csv('/content/concatenated_dataset.csv')

# Add a column for manual annotations
df['causal_relation'] = ""

# Manually check causal relations
df = manual_check_causal_relation(df)

# Save the annotated dataset
df.to_csv('annotated_dataset.csv', index=False)


Example 1:
Text: The State alleged they hacked Sabata Petros Chale , 39 , to death in Marikana West , on December 8 , 2016 , allegedly over the allocation of low cost ( RDP ) houses at Marikana West Extension 2 .
Does this example contain a causal relation? (Yes/No): Yes

Example 2:
Text: Chale was allegedly chased by a group of about 30 people and was hacked to death with pangas , axes and spears .
Does this example contain a causal relation? (Yes/No): no

Example 3:
Text: The farmworkers ' strike resumed on Tuesday when their demands were not met .
Does this example contain a causal relation? (Yes/No): no

Example 4:
Text: Demonstrators have filed for a permit to hold a rally on Saturday in Yuen Long , the district on the outskirts of Hong Kong where dozens of masked men chased and beat commuters and protesters with wooden poles and metal rods , leaving at least 45 people in hospital .
Does this example contain a causal relation? (Yes/No): yes

Example 5:
Text: Footage of the attack

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the annotated dataset
df = pd.read_csv('/content/concatenated_dataset.csv')

# Check for NaN values in the original DataFrame
print("NaN values in original DataFrame:", df.isnull().values.any())

# Drop rows with NaN values
df.dropna(inplace=True)

# Check if the dataset is empty after dropping NaN values
if len(df) == 0:
    print("Dataset is empty after dropping NaN values. Adjust the data or preprocessing steps.")
    exit()

# Fill NaN values in the 'text' column with an empty string
df['text'].fillna('', inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['causal_text_w_pairs'], test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_vec, y_train)

# Predict on the test set
y_pred = classifier.predict(X_test_vec)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Apply the classifier to the entire dataset
X_vec = vectorizer.transform(df['text'])
df['predicted_causal_relation'] = classifier.predict(X_vec)

# Save the updated dataset
df.to_csv('predicted_dataset.csv', index=False)

NaN values in original DataFrame: True
Dataset is empty after dropping NaN values. Adjust the data or preprocessing steps.


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.